imports

In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm

import pandas as pd

from myhelpers import cifar_dataLoader

from HGNN.train.configParser import ConfigParser
from HGNN.train import CNN, dataLoader
from myhelpers.memory import get_cuda_memory
from myhelpers.image_show import showExample, getClosestImageFromDataloader
from myhelpers.images_tsne import get_images_tsne
from misc import get_classification_df

experimetnsFileName = "experiments.csv"

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName="Fish_scheduler_experiments"#"CIFAR_phylogeny_HGNN_lambdaExperiment"
trial_hash="5922bad3c69f629daa0af24121e292a895f308e7cae3ec4f64536186"#hier

use_submodel=True
cuda=1
batch_size = 4
num_workers = 8

cuda

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)
    
torch.multiprocessing.set_start_method('spawn')

using cuda 1


Get dataset

In [4]:
# Get experiment parameters
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
    experiments_df = experiments_df.where(pd.notnull(experiments_df), None)
else:
    raise Exception("Experiment not " + trial_hash + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]
print(experiment_params)

config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experimentPathAndName = os.path.join(experimentsPath, experimentName)
if experiment_params['image_path'] == 'cifar-100-python':
    datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
else:
    datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

dataloader = torch.utils.data.DataLoader(test_loader.dataset, pin_memory=True, batch_size=batch_size, num_workers=num_workers)
dataset = dataloader.dataset

{'experimentName': 'Fish_scheduler_experiments', 'modelName': 'models/5922bad3c69f629daa0af24121e292a895f308e7cae3ec4f64536186', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': 'd76671f30f9c5b1773e7c876f09f3c399e6e6f05d785b8dcb5023857', 'trialHash': '5922bad3c69f629daa0af24121e292a895f308e7cae3ec4f64536186', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 1, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.0, 'unsupervisedOnTest': None, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'noSpeciesBackprop': False, 'phylogeny_loss': 'False', 'phylogeny_loss_epsilon': 0.03, 'tripletEnabled': True, 'tripletSamples': 2.0, 'tripletSelector': 'semihard', 'tripletMargin': 2.0, 'displayName': 'Fish-s-triplet-hier-gamma0.5', 'pretrained': True, 'epochs': 40.0, 'patience': -1.0, 

Get untrained model

In [5]:
%%capture

architecture = {
    "fine": len(dataset.csv_processor.getFineList()),
    "coarse" : len(dataset.csv_processor.getCoarseList())
}
model = CNN.create_model(architecture, experiment_params, cuda)

# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)
_ = CNN.loadModel(model, trialName)
model.eval()

In [6]:
predProblist_, lbllist = CNN.getLoaderPredictionProbabilities(dataloader, model, experiment_params, device=cuda)

Show correctly classified and misclassified examples

In [7]:
df_misclassified, df_correctlyclassified = get_classification_df(dataloader.dataset, predProblist_, lbllist, modelName, experimentPathAndName)
df_misclassified

100%|██████████| 380/380 [00:38<00:00,  9.96it/s]


,file name,true label,probability of true label,predicted label,original_index
2,INHS_FISH_23436.jpg,5,0.015049,6,53.0
25,INHS_FISH_41599.jpg,5,0.111350,6,56.0
38,INHS_FISH_102575.jpg,5,0.224780,6,50.0
32,INHS_FISH_7120.jpg,6,0.168613,3,67.0
29,INHS_FISH_46448.jpg,8,0.134303,7,80.0
37,INHS_FISH_55956.jpg,9,0.224730,14,98.0
0,INHS_FISH_25410.jpg,10,0.003606,12,101.0
5,INHS_FISH_106249.jpg,10,0.023604,8,100.0
19,INHS_FISH_103481.jpg,11,0.065845,13,110.0
24,INHS_FISH_92287.jpg,11,0.106653,12,117.0


In [8]:
df_correctlyclassified

,file name,true label,probability of true label,predicted label,original_index
140,INHS_FISH_106912.JPG,0,0.906782,0,4.0
216,INHS_FISH_106186.JPG,0,0.964463,0,3.0
221,INHS_FISH_103372.JPG,0,0.966780,0,1.0
241,INHS_FISH_4568.JPG,0,0.975593,0,6.0
243,INHS_FISH_38264.JPG,0,0.977289,0,5.0
...,...,...,...,...,...
255,INHS_FISH_42051.jpg,37,0.979429,37,373.0
256,INHS_FISH_18365.jpg,37,0.979893,37,371.0
278,INHS_FISH_39150.jpg,37,0.986579,37,372.0
301,INHS_FISH_73996.jpg,37,0.992462,37,375.0


Show example and closest images